In [2]:
import traceback

import polars as pl
import sys
import torch
import polars as pl
import numpy as np
from sklearn.linear_model import LinearRegression
from lifelines import WeibullFitter
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader

from utils.custom_losses import CustomLoss
from utils.lstf_feature_maker.piecewise_linear_regression import PiecewiseLinearRegression
from utils.lstf_feature_maker.weibull import WeibullFeatureMaker

'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '../data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR

tb_bas_oper_part_mst = (pl.read_parquet(DIR + 'tb_bas_oper_part_mst.parquet')
                        .select(['OPER_PART_NO', 'OPER_PART_NM'])
                        .rename({'OPER_PART_NO': 'oper_part_no', 'OPER_PART_NM': 'oper_part_nm'}))
tb_dyn_fcst_demand = (pl.read_parquet(DIR + 'tb_dyn_fcst_dmnd.parquet')
                      .select(['PART_NO', 'DMND_QTY', 'DMND_DT', 'OPER_PART_NO'])
                      .rename({'PART_NO': 'part_no', 'OPER_PART_NO': 'oper_part_no', 'DMND_DT': 'demand_dt', 'DMND_QTY': 'demand_qty'})
                      .select(['part_no', 'oper_part_no', 'demand_dt', 'demand_qty']))
tb_dyn_fcst_demand_sellout = (pl.read_parquet(DIR + 'tb_dyn_fcst_dmnd_sellout.parquet')
                              .select(['PART_NO', 'DMND_QTY', 'DMND_DT', 'OPER_PART_NO'])
                              .rename({'PART_NO': 'part_no', 'OPER_PART_NO': 'oper_part_no', 'DMND_DT': 'demand_dt', 'DMND_QTY': 'demand_qty'})
                              .select(['part_no', 'oper_part_no', 'demand_dt', 'demand_qty']))

In [3]:
from utils.date_util import DateUtil

dyn_fcst_demand = tb_dyn_fcst_demand.with_columns([
    pl.col('demand_dt').cast(pl.Int64).map_elements(DateUtil.yyyymmdd_to_date, return_dtype = pl.Date)
])

dyn_demand_sellout = tb_dyn_fcst_demand_sellout.with_columns([
    pl.col('demand_dt').cast(pl.Int64).map_elements(DateUtil.yyyymmdd_to_date, return_dtype = pl.Date)
])

dyn_fcst_demand

part_no,oper_part_no,demand_dt,demand_qty
str,str,date,f64
"""T4240-71102BB""","""T4240-71102BB""",2018-01-01,3.0
"""T5210-34402""","""T5210-34402""",2018-01-01,1.0
"""T5210-30081""","""T5210-30081""",2018-01-01,1.0
"""T5210-65661""","""T5210-65661""",2018-01-01,1.0
"""T5210-66472""","""T5210-66472""",2018-01-01,1.0
…,…,…,…
"""U3215-52203""","""U3215-52203""",2024-02-05,30.0
"""T5710-69252""","""T5710-69252""",2024-02-05,2.0
"""DYD1-O07""","""DYD1-O07""",2024-02-05,4.0


In [4]:
dyn_fcst = (dyn_fcst_demand
                .join(tb_bas_oper_part_mst, on = 'oper_part_no', how = 'left')
                .select(['oper_part_no', 'oper_part_nm', 'demand_dt','demand_qty'])
                .sort(['oper_part_no', 'demand_dt'])
                .with_columns([
                    pl.col('demand_qty').cum_sum().over('oper_part_no').alias('cumsum_qty')
                ])
              )
dyn_demand = (dyn_demand_sellout.join(tb_bas_oper_part_mst, on = 'oper_part_no', how = 'left')
                    .select(['oper_part_no', 'oper_part_nm', 'demand_dt', 'demand_qty'])
                    .sort(['oper_part_no', 'demand_dt'])
                    .with_columns([
                        pl.col('demand_qty').cum_sum().over('oper_part_no').alias('cumsum_qty')
                 ])
               )

In [8]:
from utils.lstf_feature_maker.exponential_smoothing import ExponentialSmoothing

lookback_window = 40
horizon = 20
feature_schema = {
    'oper_part_no': pl.Utf8,
    'X_ts': pl.Array(pl.Float64, 40),
    'y_ts': pl.Array(pl.Float64, 20),
    'X_features': pl.List(pl.Float64)
}

def compute_feature_group(df: pl.DataFrame) -> pl.DataFrame:
    try:
        part_no = df['oper_part_no'][0]
        series = df['demand_qty'].to_numpy()

        # lookback_window와 horizon 확인
        if len(series) < lookback_window + horizon:
            return pl.DataFrame(schema=feature_schema)

        # Feature 계산
        LTB_point = len(series) - horizon
        slope_pre, slope_saddle, slope_post = PiecewiseLinearRegression(series = series, start_point = LTB_point).auto_piecewise_slopes()
        k, lam = WeibullFeatureMaker().auto_weibull_params(series)

        # Rolling Mean, SES
        ma9 = df['demand_qty'].rolling_mean(9).to_list()
        ses = ExponentialSmoothing().simple_exponential_smoothing(series = df['demand_qty'].to_list())

        X_ts, y_ts, X_features = [], [], []
        for i in range(len(series) - lookback_window - horizon - 1):
            X_ts.append(series[i:i+lookback_window])
            y_ts.append(series[i+lookback_window:i+lookback_window+horizon])
            X_features.append([
                slope_pre, slope_saddle, slope_post, k, lam,
                ma9[i + lookback_window], ses[i + lookback_window]
            ])

        return pl.DataFrame({
            'oper_part_no': [part_no] * len(X_ts),
            'X_ts': X_ts,
            'y_ts': y_ts,
            'X_features': X_features
        })

    except Exception as e:
        print(f"Error in compute_feature_group: {e}")
        print(traceback.print_exc())
        # return pl.DataFrame(schema=feature_schema)  # 항상 동일 스키마 반환

In [10]:
df_computed_grouped = (dyn_demand
    .select(['oper_part_no', 'demand_dt', 'demand_qty', 'cumsum_qty'])
    .sort(['oper_part_no', 'demand_dt'])
    .with_columns(pl.col('demand_dt').map_elements(DateUtil.date_to_yyyymm, return_dtype = pl.Int64).alias('demand_dt'))
    .select('oper_part_no', 'demand_dt', 'demand_qty')
    .group_by(['oper_part_no', 'demand_dt'])
    .agg([
        pl.col('demand_qty').sum().alias('demand_qty')
    ])
    .sort(['oper_part_no', 'demand_dt'])
    .group_by('oper_part_no')
    .map_groups(compute_feature_group)
 )


In [21]:
# df_computed_grouped.write_parquet(DIR + '\\patch_mixer_parquets\\L_40_H_20_master.parquet')
df_computed_grouped['X_features']

X_features
list[f64]
"[-13.424869, 4.5, … 17.433615]"
"[-13.424869, 4.5, … 15.20353]"
"[-13.424869, 4.5, … 10.942471]"
"[-13.424869, 4.5, … 10.65973]"
"[-0.03005, 6.1, … 4.471545]"
…
"[0.0, 0.0, … 2687.841903]"
"[0.0, 0.0, … 2067.489332]"
"[0.0, 0.0, … 2950.542532]"


In [18]:
X_ts = np.vstack(df_computed_grouped['X_ts'].to_list())
y_ts = np.vstack(df_computed_grouped['y_ts'].to_list())
X_features = np.vstack(df_computed_grouped['X_features'].to_list())

In [22]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler_X = MinMaxScaler()
X_ts_scaled = scaler_X.fit_transform(X_ts.reshape(-1, 1)).reshape(X_ts.shape)

y_ts_log = np.log1p(y_ts)
scaler_y = MinMaxScaler()
y_ts_scaled = scaler_y.fit_transform(y_ts_log.reshape(-1, 1)).reshape(y_ts_log.shape)

scaler_feat = StandardScaler()
X_features_scaled = scaler_feat.fit_transform(X_features)

In [23]:
class DemandDataset(Dataset):
    def __init__(self, X_ts, X_feat, y_ts):
        self.X_ts = torch.tensor(X_ts, dtype = torch.float32).unsqueeze(-1)
        self.X_feat = torch.tensor(X_feat, dtype = torch.float32)
        self.y_ts = torch.tensor(y_ts, dtype = torch.float32)

    def __len__(self):
        return len(self.X_ts)

    def __getitem__(self, idx):
        return self.X_ts[idx], self.X_feat[idx], self.y_ts[idx]

X_train, X_val, F_train, F_val, y_train, y_val = train_test_split(X_ts_scaled, X_features_scaled, y_ts_scaled, test_size=0.2, random_state=42)


train_dataset = DemandDataset(X_train, F_train, y_train)
val_dataset = DemandDataset(X_val, F_val, y_val)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)

In [25]:
from models.PatchMixer import PatchMixerFeatureModel
class Config:
    enc_in = 1
    seq_len = lookback_window
    pred_len = horizon
    patch_len = 16
    stride = 8
    mixer_kernel_size = 8
    d_model = 16
    head_dropout = 0.1
    e_layers = 2

config = Config()

if sys.platform == 'win32':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
else:
    device = torch.device('mps' if torch.mps.is_available() else 'cpu')

model = PatchMixerFeatureModel(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3, weight_decay = 1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 50)
max_grad_norm = 1.0
patience = 30
best_loss = float('inf')
counter = 0
best_model_path = 'best_model_20250804.pth'
num_epoch = 5000

In [ ]:
for epoch in range(num_epoch):
    model.train()
    total_train_loss = 0.0

    for X_ts_batch, X_feat_batch, y_batch in train_loader:
        X_ts_batch, X_feat_batch, y_batch = X_ts_batch.to(device), X_feat_batch.to(device), y_batch.to(device)

        pred = model(X_ts_batch, X_feat_batch)
        loss = CustomLoss(pred, y_batch).combined_loss()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = max_grad_norm)
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    model.eval()
    total_val_loss = 0.0

    with torch.no_grad():
        for X_ts_batch, X_feat_batch, y_batch in val_loader:
            X_ts_batch, X_feat_batch, y_batch = X_ts_batch.to(device), X_feat_batch.to(device), y_batch.to(device)

            pred = model(X_ts_batch, X_feat_batch)
            val_loss = CustomLoss(pred, y_batch).combined_loss()
            total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(val_loader)
        scheduler.step()
        print(f"Epoch [{epoch + 1}/{num_epoch}], Train Loss: {avg_train_loss:.8f}, Val Loss: {avg_val_loss:.8f}")

        # Early Stopping
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            counter = 0
            torch.save(model.state_dict(), best_model_path)

        else:
            counter += 1
            if counter >= patience:
                print('Early Stopping Triggered')
                break

model.load_state_dict(torch.load(best_model_path))